In [1]:
%run load_actiwatch_data.py
%run firsttime.py

import matplotlib.pyplot as plt
%matplotlib inline

import pyarrow

from joblib import *

import statsmodels.api as sm
import statsmodels.formula.api as smf

import scipy.stats as stats

# this is used to make Federal Holidays a nonschool day.  Note that we don't have any
# way to recognize school district unique holidays, like teacher work days or such 
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar


In [2]:
# change the values in this cell to process the data desired

global ddirs

# dictionary links a unique experimental group with the directory that holds those files
# inside that directory are unique files per subject in the group
ddirs = {
        'Fall 220 2015': '../Fall 220 2015/',
        'Fall 220 2016': '../Fall 220 2016/',
        'Spring 418 2016': '../Spring 418 2016/',
        'Spring 418 2017': '../Spring 418 2017/',
        'Spring 418 2018': '../Spring 418 2018/',
        'Summer 220 2016': '../Summer 220 2016/',
        'Summer 220 2017': '../Summer 220 2017/',
        'Winter 220 2018': '../Winter 220 2018/',
}

# where the saved processed data goes
outfile = '../processed data/SeaUgrad'

In [3]:
recalculate_raw = False # true forces long recalculations, false loads processed data from disk
recalculate_timing = False

threshs = [ [5], [10], [50], [100], [500], [1000] ] 

def get_raw_data(season):
    rawd, summaryd = load_actiwatch_data(ddirs[season],uidprefix=season)
    rawd['Group']=season
    return rawd    

if recalculate_raw:
    print("Loading raw from disk ...")    
    results = Parallel(n_jobs=len(ddirs))(delayed(get_raw_data)(season) for season in ddirs.keys())
    allData = pd.concat(results)
    # the following assignments depend on the datafile names and directory structure being exactly of the form:
    # ../Quarter Class 4digitYear/uniqueSubjectID_blahblahwhatever.csv
    allData['Quarter']=allData.UID.apply(lambda x: x.split()[0])
    allData['Class']=allData.UID.apply(lambda x: x.split()[1])
    allData['Year']=allData.UID.apply(lambda x: x.split()[2][:4])
    allData['Subject ID']=allData.UID.apply(lambda x: x.split()[2][4:])
    allData.to_parquet(outfile+'raw.parquet',engine='fastparquet',compression='gzip')
else:
    allData = pd.read_parquet(outfile+'raw.parquet')
    
if recalculate_timing:    
    print("Calculating light timing data ...")
    
    # don't recalculate results if it already exists, takes a long time and a lot of memory
    # this has been added as I troubleshoot later manipulations in this cell... should be ok
    # if we are recalculating from scratch but this might bite you in the butt if there's some
    # weird non-linear cell execution going on with another variable named results
    try: results
    except NameError:
        results = Parallel(n_jobs=len(threshs))(delayed(firstAndLastLight)(allData, threshold) for threshold in threshs)
        
    timingData = pd.concat(results)
    
    print("Adding holiday markers to timing data ...")
    cal = calendar()
    holidays = cal.holidays(start=timingData.Date.min(), end=timingData.Date.max())

    nn = pd.DatetimeIndex( timingData.Date )
    timingData['DayofWeek'] = nn.dayofweek 
    days = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']
    daytype = ['Weekday','Weekday','Weekday','Weekday','Weekday','Weekend/Holiday','Weekend/Holiday']

    daygrp=[]
    dtpgrp=[]
    wkendholiday=[]
    for k, row in timingData.iterrows():
        daygrp.append(row['Group'].split('seattle')[0] + days[row['DayofWeek']])
        if holidays.isin([row['Date']]).any():
            dtpgrp.append(row['Group'].split('seattle')[0] + 'Weekend/Holiday')
            wkendholiday.append(True)
        else:
            dtpgrp.append(row['Group'].split('seattle')[0] + daytype[row['DayofWeek']])
            if row['DayofWeek'] > 4:
                wkendholiday.append(True)
            else:
                wkendholiday.append(False)

    timingData['GroupDayofWeek'] = daygrp
    timingData['GroupDayType'] = dtpgrp
    timingData['Weekend/Holiday'] = wkendholiday

    # missing stuff was here
    
    timingData['Quarter']=timingData.UID.apply(lambda x: x.split()[0])
    timingData['Class']=timingData.UID.apply(lambda x: x.split()[1])
    timingData['Year']=timingData.UID.apply(lambda x: x.split()[2][:4])
    timingData['Subject ID']=timingData.UID.apply(lambda x: x.split()[2][4:])

    #timingData.to_parquet(outfile+'timing.parquet')
    # at this moment there are two choices for parquet writing, pyarrow (which does not support Timedelta)
    # and fastparquet (which does not support datetime.date )... I'm going to go with the latter
    tdc = timingData.copy()
    tdc.Date = tdc.Date.astype('datetime64')
    tdc['Watch period'] = pd.to_timedelta(tdc['Watch period']) #oops that was a datetime.timedelta, which we dont need that also choked parquet
    tdc.to_parquet(outfile+'timing.parquet', engine='fastparquet', compression='gzip')
    del(tdc)
else:
    tdc = pd.read_parquet(outfile+'timing.parquet', engine='fastparquet')
    tdc.Date = tdc.Date.apply(lambda x: x.date()) # go back to original data format
    timingData = tdc.copy()

    

Calculating light timing data ...
Adding holiday markers to timing data ...


In [6]:
touse = pd.read_excel('../Subjects Included in Study.xlsx')
checkme = touse[['Quarter','Year','Class','Subject ID']].drop_duplicates().sort_values(by=['Quarter', 'Year','Class','Subject ID'])
print(checkme.dtypes)
checkme

Quarter       object
Year           int64
Class          int64
Subject ID     int64
dtype: object


,Quarter,Year,Class,Subject ID
0,Fall,2015,220,1
1,Fall,2015,220,2
2,Fall,2015,220,3
3,Fall,2015,220,5
4,Fall,2015,220,6
...,...,...,...,...
438,Winter,2018,220,116
439,Winter,2018,220,117
440,Winter,2018,220,118
441,Winter,2018,220,119


In [7]:
dataids = timingData[['Quarter','Year','Class','Subject ID']].drop_duplicates()
dataids['Subject ID'] = dataids['Subject ID'].astype(int)
dataids['Year'] = dataids['Year'].astype(int)
dataids['Class'] = dataids['Class'].astype(int)
dataids.sort_values(by=['Quarter', 'Year','Class','Subject ID'],inplace=True)
dataids.reset_index(drop=True,inplace=True)
print(dataids.dtypes)
dataids

Quarter       object
Year           int64
Class          int64
Subject ID     int64
dtype: object


,Quarter,Year,Class,Subject ID
0,Fall,2015,220,1
1,Fall,2015,220,2
2,Fall,2015,220,3
3,Fall,2015,220,5
4,Fall,2015,220,6
...,...,...,...,...
504,Winter,2018,220,116
505,Winter,2018,220,117
506,Winter,2018,220,118
507,Winter,2018,220,119


In [8]:
df_all = pd.concat( [dataids, checkme])
rmall = df_all.drop_duplicates(keep=False, inplace=False)
rmall['UID']=rmall.apply(axis=1, func=lambda x: '{} {} {}{:04d}'.format(x['Quarter'],x['Class'],x['Year'],x['Subject ID']))
rmall

/Users/jfleischer/opt/anaconda3/envs/light_analysis_2020/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,Quarter,Year,Class,Subject ID,UID
463,Winter,2018,220,57,Winter 220 20180057
503,Winter,2018,220,115,Winter 220 20180115


In [9]:
rmweekends = touse.loc[ touse.Notes.dropna().index, ['Quarter','Year','Class','Subject ID'] ]
rmweekends['UID']=rmweekends.apply(axis=1, func=lambda x: '{} {} {}{:04d}'.format(x['Quarter'],x['Class'],x['Year'],x['Subject ID']))
rmweekends

,Quarter,Year,Class,Subject ID,UID
67,Fall,2015,220,79,Fall 220 20150079
93,Fall,2015,220,114,Fall 220 20150114
200,Summer,2016,220,49,Summer 220 20160049
216,Summer,2016,220,65,Summer 220 20160065
229,Summer,2016,220,80,Summer 220 20160080
232,Fall,2016,220,1,Fall 220 20160001
244,Fall,2016,220,17,Fall 220 20160017
255,Fall,2016,220,30,Fall 220 20160030
323,Spring,2017,418,62,Spring 418 20170062
397,Winter,2018,220,55,Winter 220 20180055


In [10]:
#remove subjects who don't belong
thedata = allData.copy()
thetiming = timingData.copy()
print(thedata.shape, thetiming.shape)
for anid in rmall.UID:
    thedata = thedata.query('~(UID == @anid)')
    thetiming = thetiming.query('~(UID == @anid)')
print(thedata.shape,  thetiming.shape)

(43664740, 15) (47844, 23)
(43544129, 15) (47712, 23)


In [11]:
thedata['is_weekend'] = thedata.index.dayofweek.isin([5,6])
thedata['is_weekend'].describe()

count     43544129
unique           2
top          False
freq      31506511
Name: is_weekend, dtype: object

In [12]:
# ratio of weekday to total datapoints is close ot the expected values
print(31.5/43.5, 5./7.)

0.7241379310344828 0.7142857142857143


In [13]:
thetiming['OutofSchool'] = thetiming['Weekend/Holiday']

In [14]:
# remove weekend dates for subjects who shouldn't be in the weekend data
print(thedata.shape, thetiming.shape)
for anid in rmweekends.UID:
    thedata = thedata.query('~((UID == @anid)&(is_weekend == True))')
    thetiming = thetiming.query('~((UID == @anid)&(OutofSchool == True))')
print(thedata.shape, thetiming.shape)

(43544129, 16) (47712, 24)
(43244609, 16) (47430, 24)


In [15]:
thedata['is_weekend'].describe()

count     43244609
unique           2
top          False
freq      31506511
Name: is_weekend, dtype: object

In [16]:
# ratio is still good
31.5/43.2

0.7291666666666666

In [17]:
# OK calculate on only the good data
allData=thedata

In [18]:
allddate= pd.Series( allData.index.date )
holidays = calendar().holidays(start=timingData.Date.min(), end=timingData.Date.max())
allData['is_holiday'] = allddate.apply( lambda x: holidays.isin([x]).any())
allData['OutofSchool'] = allData['is_weekend'] | allData['is_holiday']

In [19]:
allData['Group'].unique()

array(['Fall 220 2015', 'Fall 220 2016', 'Spring 418 2016',
       'Spring 418 2017', 'Spring 418 2018', 'Summer 220 2016',
       'Summer 220 2017', 'Winter 220 2018'], dtype=object)

In [20]:
dpart = allData['OutofSchool'].apply(lambda x: 'Non-school day' if x else 'School day')
allData['GroupDayType'] = allData['Group'].str.cat( dpart, sep=' ')
allData['UIDDayType'] = allData['UID'].str.cat( dpart, sep=' ')

In [21]:
def hours_float( dttm ):
    # takes timestamp, returns floating point hours description of the time  
    if pd.isna(dttm):
        return np.NaN
    
    td = pd.Timedelta( dttm.time().isoformat() )
    return td.total_seconds() / 3600. 

hours_float( thetiming['First Light'].iloc[0] )

5.920833333333333

In [22]:
thetiming['firstlight']=thetiming['First Light'].apply(lambda x: hours_float(x))
thetiming['lastlight']=thetiming['Last Light'].apply(lambda x: hours_float(x))
thetiming['lastlight']=thetiming['lastlight'].apply( lambda x: x if x>4.0 else x+24.)

In [23]:
thetiming.Date.unique().shape

(271,)

In [24]:
from astral import *
a = Astral()

dawns=[]
dusks=[]
dlookup = []
dates = thetiming.Date.unique()

for day in dates:
    dlookup.append( a['Seattle'].sun(date=day) )
    
lookupday = pd.Series(dlookup,index=dates)
 
print('Sunset calc')
thetiming['Sunset'] = thetiming.Date.apply( lambda x: hours_float( lookupday[x]['sunset'] ) )
print('Sunrise calc')
thetiming['Sunrise'] = thetiming.Date.apply( lambda x: hours_float( lookupday[x]['sunrise'] ) )

Sunset calc
Sunrise calc


In [25]:
thetiming['hours from sunrise to first light abv threshold'] = thetiming['firstlight'] - thetiming['Sunrise']
thetiming['hours from sunset to last light abv threshold'] = thetiming['lastlight'] - thetiming['Sunset']

In [26]:
thetiming.GroupDayType = thetiming.GroupDayType.apply(lambda x: x.replace('Weekday',' School day').replace('Weekend/Holiday',' Non-school day'))

In [27]:
thetiming.GroupDayType = thetiming.GroupDayType.apply(lambda x: x.replace(' School day','\nSchool day').replace(' Non-school day','\nNon-school day'))

In [28]:
# there was some accidental duplication the first time I was working out this analysis, won't be necessary in the future as I fixed the problem
thetiming = thetiming.loc[:,~thetiming.columns.duplicated() ]
thetiming.Date = thetiming.Date.astype('datetime64')
thetiming['Watch period'] = pd.to_timedelta(thetiming['Watch period'])
thetiming.to_parquet(outfile+'TimingAnalysis.parquet', engine='fastparquet', compression='gzip')

In [29]:
result1 = thetiming.query('Threshold == 50').groupby(['UID','OutofSchool'])['firstlight'].describe()
result1.unstack().to_excel('../processed data/Time to 1st 50lux per person.xlsx')

In [40]:
result2 = thetiming.query('Threshold == 50').groupby(['UID','OutofSchool'])['Minutes above threshold'].describe()
result2.unstack().to_excel('../processed data/Minutes above 50lux per person.xlsx')

In [41]:
result3 = thetiming.query('Threshold == 50').groupby(['UID','OutofSchool'])['Minutes above threshold AM'].describe()
result3.unstack().to_excel('../processed data/Minutes above 50lux per person in the AM.xlsx')

In [42]:
result4 = thetiming.query('Threshold == 5').groupby(['UID','OutofSchool'])['Lux minutes'].describe()
result4.unstack().to_excel('../processed data/Total lux minutes per person.xlsx')

In [43]:
result5 = thetiming.query('Threshold == 5').groupby(['UID','OutofSchool'])['Lux minutes AM'].describe()
result5.unstack().to_excel('../processed data/Total lux minutes per person in the AM.xlsx')

In [50]:
( thetiming.query('Threshold == 50')[['UID','Date','OutofSchool','firstlight']]
     .set_index(['UID','Date']).sort_index()
     .to_excel('../processed data/Per person-day time to 1st 50lux.xlsx')
)

In [51]:
( thetiming.query('Threshold == 50')[['UID','Date','OutofSchool','Minutes above threshold']]
     .set_index(['UID','Date']).sort_index()
     .to_excel('../processed data/Per person-day minutes above 50lux.xlsx')
)

In [52]:
( thetiming.query('Threshold == 50')[['UID','Date','OutofSchool','Minutes above threshold AM']]
     .set_index(['UID','Date']).sort_index()
     .to_excel('../processed data/Per person-day time minutes above 50lux in the AM.xlsx')
)

In [53]:
( thetiming.query('Threshold == 5')[['UID','Date','OutofSchool','Lux minutes']]
     .set_index(['UID','Date']).sort_index()
     .to_excel('../processed data/Per person-day total lux min.xlsx')
)

In [54]:
( thetiming.query('Threshold == 5')[['UID','Date','OutofSchool','Lux minutes AM']]
     .set_index(['UID','Date']).sort_index()
     .to_excel('../processed data/Per person-day total lux min in the AM.xlsx')
)